In [10]:
from os import listdir
from os.path import isfile, join
train_path_ = '/home/faiq/Study/HLCV/Project/Phase 2/Exemplar_GAN/output/sample0/sample_Experiment_6_21_6'
test_path_ = '/home/faiq/Study/HLCV/Project/Phase 2/Exemplar_GAN/output/sample1/sample_Experiment_6_21_6'
final_path = '/home/faiq/Study/HLCV/Project/Phase 2/Exemplar_GAN/output/Final_images'
final_train_path = '/home/faiq/Study/HLCV/Project/Phase 2/Exemplar_GAN/output/Final_train_images'
trainfiles = [train_path_+"/"+f for f in listdir(train_path_) if isfile(join(train_path_, f))]
testfiles = [test_path_+"/"+f for f in listdir(test_path_) if isfile(join(test_path_, f))]

In [20]:
import cv2
def get_err_test(image, i):
    crop_img_gt1 = img[:int(img.shape[0]/4), :int(img.shape[1]/4)]
    crop_img_gt2 = img[:int(img.shape[0]/4), int(img.shape[1]/4*3):]
    crop_img_pred1 = img[int(img.shape[0]/4*3):, :int(img.shape[1]/4)]
    crop_img_pred2 = img[int(img.shape[0]/4*3):, int(img.shape[1]/4*3):]
    cv2.imwrite("{}/{}_{}.jpg".format(final_path, str(i), '0'), crop_img_gt1)
    cv2.imwrite("{}/{}_{}.jpg".format(final_path, str(i), '1'), crop_img_pred1)
    cv2.imwrite("{}/{}_{}.jpg".format(final_path, str(i+1), '0'), crop_img_gt2)
    cv2.imwrite("{}/{}_{}.jpg".format(final_path, str(i+1), '1'), crop_img_pred2)
#     cv2.imshow("Display window", crop_img_gt1)
#     k = cv2.waitKey(0)
#     cv2.imshow("Display window", crop_img_pred1)
#     k = cv2.waitKey(0)
#     cv2.imshow("Display window", crop_img_gt2)
#     k = cv2.waitKey(0)
#     cv2.imshow("Display window", crop_img_pred2)
#     k = cv2.waitKey(0)
def get_err_train(image, i):
    crop_img_gt1 = img[0:int(img.shape[0]/6), :int(img.shape[1]/4)]
    crop_img_gt2 = img[0:int(img.shape[0]/6), int(img.shape[1]/4*3):]
    crop_img_pred1 = img[int(img.shape[0]/6*3):int(img.shape[0]/6*4), :int(img.shape[1]/4)]
    crop_img_pred2 = img[int(img.shape[0]/6*3):int(img.shape[0]/6*4), int(img.shape[1]/4*3):]
#     print(final_train_path, str(i)+"_1", '0')
    cv2.imwrite("{}/{}_{}.jpg".format(final_train_path, str(i)+"_1", '0'), crop_img_gt1)
    cv2.imwrite("{}/{}_{}.jpg".format(final_train_path, str(i)+"_1", '1'), crop_img_pred1)
    cv2.imwrite("{}/{}_{}.jpg".format(final_train_path, str(i)+"_2", '0'), crop_img_gt2)
    cv2.imwrite("{}/{}_{}.jpg".format(final_train_path, str(i)+"_2", '1'), crop_img_pred2)

In [22]:
i = 1
for file in trainfiles:
    img = cv2.imread(file)
    get_err_train(img, file.split("/")[-1][:-4])
#     break

In [31]:
 
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))

2022-07-31 02:51:13.956231: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-31 02:51:13.956281: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pringles): /proc/driver/nvidia/version does not exist
2022-07-31 02:51:13.957293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# example of calculating the frechet inception distance in Keras
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from collections import defaultdict

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 
# prepare the inception v3 model
# model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
# define two fake collections of images
FID_dict = defaultdict(lambda x: "Not Present")
finalfiles = [final_path+"/"+f for f in listdir(final_path) if isfile(join(final_path, f))]
total_images = int(len(finalfiles)/2)
for f in sorted(listdir(final_train_path)):
    if str(f[-5:-4]) == '1':
        continue
    print("Image1: ", f)
    print("Image2: ", f[:-6]+"_1.jpg")
    images1 = cv2.imread(final_train_path+"/"+f)
    images2 = cv2.imread(final_train_path+"/"+f[:-6]+"_1.jpg")
    print('Prepared', images1.shape, images2.shape)
    # convert integer to floating point values
    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    # resize images
    images1 = scale_images(images1, (256,256,3))
    images2 = scale_images(images2, (256,256,3))
    print('Scaled', images1.shape, images2.shape)
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    # fid between images1 and images2
    fid = calculate_fid(model, images1, images2)
    print('FID ({} image): {:.3f}'.format(i, fid))
    FID_dict[f] = fid

Image1:  00_output_0.jpg
Image2:  00_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 8s 962ms/step
FID (1 image): 200.154
Image1:  10000_output_0.jpg
Image2:  10000_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 8s 991ms/step
FID (1 image): 26.138
Image1:  10400_output_0.jpg
Image2:  10400_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 8s 996ms/step
FID (1 image): 36.212
Image1:  10800_output_0.jpg
Image2:  10800_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 8s 1s/step
FID (1 image): 29.499
Image1:  11200_output_0.jpg
Image2:  11200_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [=============

8/8 [==============================] - 9s 1s/step
FID (1 image): 70.137
Image1:  14800_output_0.jpg
Image2:  14800_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 10s 1s/step
FID (1 image): 90.109
Image1:  14900_output_0.jpg
Image2:  14900_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 60.583
Image1:  15000_output_0.jpg
Image2:  15000_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 23.674
Image1:  15100_output_0.jpg
Image2:  15100_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 51.881
Image1:  15200_output_0.jpg
Image2:  15200_output_1.jpg
Prepared (256, 256, 3) (256, 25

Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 14s 2s/step
FID (1 image): 38.236
Image1:  17900_output_0.jpg
Image2:  17900_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 23.677
Image1:  18000_output_0.jpg
Image2:  18000_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 77.224
Image1:  18100_output_0.jpg
Image2:  18100_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 24.517
Image1:  18200_output_0.jpg
Image2:  18200_output_1.jpg
Prepared (256, 256, 3) (256, 256, 3)
Scaled (256, 256, 256, 3) (256, 256, 256, 3)
8/8 [==============================] - 9s 1s/step
FID (1 image): 49.106
Image1:  18300_output_0.jpg
Image2:  18300

In [34]:
FID_dict

defaultdict(<function __main__.<lambda>(x)>,
            {'00_output_0.jpg': 200.15370825030726})

In [ ]:
# calculate inception score for cifar-10 in Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# convert from uint8 to float32
		subset = subset.astype('float32')
		# scale images to the required size
		subset = scale_images(subset, (299,299,3))
		# pre-process images, scale to [-1,1]
		subset = preprocess_input(subset)
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std

# load cifar10 images

# shuffle images
shuffle(images)
print('loaded', images.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)